In [1]:
import pandas as pd
import requests
from sp2ts import sp2dt
import xmltodict
import numpy as np

In [2]:
# Define API key
api_key = "441ni983w8egwzv"

In [3]:
# Interconnector map to rename columns in the bmrs outputs appropriately
interconnector_map = {
    'intfrGeneration':'IFA',
    'intirlGeneration':'Moyle', 
    'intnedGeneration':'BritNEd', 
    'intewGeneration':'EastWest',
    'intnemGeneration':'NEMO', 
    'intifa2Generation':'IFA2',
    'intnslGeneration':'NorthSeaLink'
}

In [4]:
interconnector_map.values()

dict_values(['IFA', 'Moyle', 'BritNEd', 'EastWest', 'NEMO', 'IFA2', 'NorthSeaLink'])

In [5]:
def format_url(**kwargs):
        '''
        Function that takes bmrs parameters and formats the url appropriately
        Parameters:
            param (* = non-mandatory)   sample data                                                         format
            version_no                  3.1
            api_key                     AP8DA23
            settlement_date*            2014-02-01                                                          YYYY-MM-DD
            settlement_period*          12                                                                  1 to 50 or *
            service_type*               csv/xml/CSV/XML
        Returns:
            url - formatted url taking into account params sent in kwargs
        '''
        url = "https://api.bmreports.com/BMRS/INTERFUELHH/{version_no}?APIKey={api_key}&FromDate={start_date}&ToDate={end_date}"
        version_no = "v1"
        kwargs['version_no'] = version_no
        if 'service_type' in kwargs:
            url = url+"&ServiceType={service_type}"
        url = url.format(**kwargs)
        return url

In [6]:
# Requesting bmrs interconnector API and renaming columns appropriately using the interconnector map
def systemDemandData(start_date, end_date):
    url = format_url(start_date=start_date, end_date=end_date, api_key=api_key)
    data = requests.get(url)
    data = xmltodict.parse(data.text)
    status = data.get('response',{}).get('responseMetadata',{}).get('httpCode',404)
    if int(status) == 200:
        data = data.get('response',{}).get('responseBody',{}).get('responseList',{}).get('item')
        df = pd.DataFrame(data)
#         print(df)
        df.rename({'startTimeOfHalfHrPeriod':'settlementPeriod'}, axis=1, inplace=True)
        df.rename(interconnector_map, axis=1, inplace=True)
        return df[['settlementDate','settlementPeriod', 'IFA', 'Moyle', 'BritNEd', 'EastWest', 'NEMO', 'IFA2', 'NorthSeaLink']]

In [7]:
# Converting settlement date and settlement period to datetime
def convert_dt(settlementDate, settlmentPeriod):
    return sp2dt(settlementDate, settlmentPeriod, closed="right")

In [8]:
start_date = '2022-08-01'
end_date = '2022-08-02'

In [9]:
df = systemDemandData(start_date, end_date)
# Converting settlementDate to pandas datetime objects
df.loc[:,'settlementDate'] = pd.to_datetime(df['settlementDate'])
# Converting settlementPeriod to integer
df.loc[:,'settlementPeriod'] = df['settlementPeriod'].astype(int)
df.loc[:,'datetime'] = df.apply(lambda x: convert_dt(x['settlementDate'],x['settlementPeriod']), axis=1)
# Filtering out only entries that are within the time period in consideration
df = df[(df['datetime'] > start_date)&(df['datetime'] <= end_date)]

/Users/kalpitha/miniforge3/envs/enc/lib/python3.9/site-packages/sp2ts/sp2ts.py:114: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if date_ < min_date or date_ > max_date:


In [10]:
# Dropping columns settlement date and settlement period 
df.drop(['settlementDate','settlementPeriod'],axis=1, inplace=True)

numeric_columns = set(df.columns) - {'datetime'}
numeric_columns = list(numeric_columns)
for i in numeric_columns:
    if i != 'datetime':
        df.loc[:, i] = df[i].astype(float)
df.loc[:, 'Total Export (MW)'] = df[df[numeric_columns] < 0].sum(axis=1)
df.loc[:, 'Total Import (MW)'] = df[df[numeric_columns] > 0].sum(axis=1)

In [12]:
# Writing dataframe as a csv file
df.to_csv('interconnector_exchanges.csv', index=False)